In [2]:
from requests import get
from bs4 import BeautifulSoup
import csv
import pandas as pd

### Get page

In [3]:
show_list = get('https://www.imdb.com/chart/toptv/')
html = BeautifulSoup(show_list.text, 'html.parser')

page = html.find("div", attrs={"class": "article"}) 

text_file = open("data.txt", "w")
n = text_file.write(str(page))
text_file.close()

### Get list of tv shows

In [4]:
pattern = '/title/tt'
  
# opening a text file
file = open("data.txt", "r")

links = []
  
# Loop through the file line by line
for line in file:  
    # checking string is present in line or not
    if pattern in line and "<img" in line:
        counter = 0
        string = ""
        for x in line:
            if x == '/':
                counter+=1
            if counter == 2:
                string += x
            if counter == 3:
                break   
        links.append(string.replace("/",""))



res = []
[res.append(x) for x in links if x not in res]

# closing text file    
file.close() 
res

['tt5491994',
 'tt0903747',
 'tt0795176',
 'tt0185906',
 'tt7366338',
 'tt0306414',
 'tt0417299',
 'tt6769208',
 'tt2395695',
 'tt0141842',
 'tt9253866',
 'tt0081846',
 'tt0944947',
 'tt2861424',
 'tt0071075',
 'tt8420184',
 'tt1355642',
 'tt1475582',
 'tt1533395',
 'tt0052520',
 'tt1877514',
 'tt12392504',
 'tt11126994',
 'tt0103359',
 'tt2560140',
 'tt0303461',
 'tt0386676',
 'tt1806234',
 'tt0296310',
 'tt9735318',
 'tt0877057',
 'tt2356777',
 'tt0098904',
 'tt2092588',
 'tt0081912',
 'tt2802850',
 'tt10541088',
 'tt7920978',
 'tt0098769',
 'tt2098220',
 'tt0092337',
 'tt0213338',
 'tt0108778',
 'tt7137906',
 'tt1865718',
 'tt3530232',
 'tt2297757',
 'tt4742876',
 'tt0063929',
 'tt2571774',
 'tt0472954',
 'tt1508238',
 'tt0081834',
 'tt0264235',
 'tt0193676',
 'tt0112130',
 'tt0072500',
 'tt3032476',
 'tt2085059',
 'tt2707408',
 'tt0098936',
 'tt0200276',
 'tt0096548',
 'tt2442560',
 'tt7660850',
 'tt3398228',
 'tt0214341',
 'tt0121220',
 'tt0353049',
 'tt1831164',
 'tt0121955',
 't

### Initialize a data frame

In [5]:
# Initializing the series that the loop will populate


### Function to get data for one episode

In [58]:
def getEpisodeData(episodes,sn,show,show_name,show_data):
    # Get the info of each episode on the page
          show = show
          season = sn
          episode_number = episodes.meta['content']
          title = episodes.a['title']
          if episodes.find('div', class_='airdate') is not None:
              airdate = episodes.find('div', class_='airdate').text.strip()
          else: airdate = None
          if episodes.find('span', class_='ipl-rating-star__rating') is not None:
             rating = episodes.find('span', class_='ipl-rating-star__rating').text
          else: rating = None
          if episodes.find('span', class_='ipl-rating-star__total-votes') is not None:
             total_votes = episodes.find('span', class_='ipl-rating-star__total-votes').text
          else: total_votes = None

          # Compiling the episode info
          episode_data = {'ID' : show,'ShowName': show_name,'Season' : season,'Episode': episode_number,'Name': title,
                          'Date' :airdate,'Rating' : rating,'Votes' : total_votes}
          # Append the episode info to the complete dataset
          show_data = show_data.append(episode_data, ignore_index = True)
          return show_data 

### Function to get data for one season

In [66]:
def getShowSeasonData(sn,show,show_data):
    # Request from the server the content of the web page by using get(),
    #and store the server’s response in the variable response
      response = get('https://imdb.com/title/'+show+'/episodes?season=' + str(sn))
      
      # Parse the content of the request with BeautifulSoup
      page_html = BeautifulSoup(response.text, 'html.parser')

      # Select all the episode containers from the season's page
      episode_containers = page_html.find_all('div', class_ = 'info')
      #show_name = page_html.find("h3", itemprop="name").text.replace('\n', ' ')
      show_name = page_html.find("h3", itemprop="name").find("a", href=True).text
      # For each episode in each season
      for episodes in episode_containers:
           show_data = getEpisodeData(episodes,sn,show,show_name,show_data)
      return show_data

In [70]:
def getDataForShow(seasons,show):
    try:
      headers = {'ID':[],
                 'ShowName': [],
                  'Season':[],
                  'Episode':[],
                  'Name':[],
                  'Date':[],
                  'Rating':[],
                  'Votes':[]
      }
      show_data = pd.DataFrame(headers)

      # For every season in the series-- range depends on the show
      for sn in range(1,seasons+1):
          show_data = getShowSeasonData(sn,show,show_data)
      return show_data
    except:
      print("Something went wrong")

### Function to get data for a range of tv shows

In [87]:
#Indexes of shows to be traversed   
def getAllTvShows():
  headers = {'ID':[],
                 'ShowName': [],
                  'Season':[],
                  'Episode':[],
                  'Name':[],
                  'Date':[],
                  'Rating':[],
                  'Votes':[]
  }

  data = pd.DataFrame(headers)


  for x in res[0:245]:
      show = x
      temp = get('https://imdb.com/title/'+show)
      html_ = BeautifulSoup(temp.text, 'html.parser')
      selector = html_.find('select', {"class": 'ipc-simple-select__input',
                                              "id" : "browse-episodes-season"})
      if selector is not None:
          seasons = int(html_.find('select', {"class": 'ipc-simple-select__input',
                                              "id" : "browse-episodes-season"}).find_all("option")[1].text)
      else:
          seasons = 1
      print(show)
      if seasons >1: 
        data = pd.concat([data, getDataForShow(seasons, show)],ignore_index=True)
  return data

In [86]:
getAllTvShows()

tt5491994
tt0903747
tt0795176
tt0185906
tt7366338
tt0306414
tt0417299
tt6769208
tt2395695
tt0141842


,ID,ShowName,Season,Episode,Name,Date,Rating,Votes
0,tt0903747,Breaking Bad,1.0,1,Pilot,2 Jan. 2012,9.0,"(31,524)"
1,tt0903747,Breaking Bad,1.0,2,Cat's in the Bag...,3 Jan. 2012,8.6,"(23,052)"
2,tt0903747,Breaking Bad,1.0,3,...And the Bag's in the River,4 Jan. 2012,8.7,"(22,368)"
3,tt0903747,Breaking Bad,1.0,4,Cancer Man,5 Jan. 2012,8.2,"(21,612)"
4,tt0903747,Breaking Bad,1.0,5,Gray Matter,6 Jan. 2012,8.3,"(21,229)"
...,...,...,...,...,...,...,...,...
265,tt0141842,The Sopranos,6.0,17,Walk Like a Man,6 May 2007,8.8,"(4,811)"
266,tt0141842,The Sopranos,6.0,18,Kennedy and Heidi,10 May 2008,9.1,"(5,709)"
267,tt0141842,The Sopranos,6.0,19,The Second Coming,10 May 2008,9.2,"(5,339)"
268,tt0141842,The Sopranos,6.0,20,The Blue Comet,17 May 2008,9.6,"(6,603)"


In [88]:
df = getAllTvShows()
df.to_csv("250shows.csv")

tt5491994
tt0903747
tt0795176
tt0185906
tt7366338
tt0306414
tt0417299
tt6769208
tt2395695
tt0141842
tt9253866
tt0081846
tt0944947
tt2861424
tt0071075
tt8420184
tt1355642
tt1475582
tt1533395
tt0052520
tt1877514
tt12392504
tt11126994
tt0103359
tt2560140
tt0303461
tt0386676
tt1806234
tt0296310
tt9735318
tt0877057
tt2356777
tt0098904
tt2092588
tt0081912
tt2802850
tt10541088
tt7920978
tt0098769
tt2098220
tt0092337
tt0213338
tt0108778
tt7137906
tt1865718
tt3530232
tt2297757
tt4742876
tt0063929
tt2571774
tt0472954
tt1508238
tt0081834
tt0264235
tt0193676
tt0112130
tt0072500
tt3032476
tt2085059
tt2707408
tt0098936
tt0200276
tt0096548
tt2442560
tt7660850
tt3398228
tt0214341
tt0121220
tt0353049
tt1831164
tt0121955
tt0384766
tt10986410
tt9432978
tt8111088
tt0074006
tt4508902
tt13991232
tt3718778
tt0367279
tt5753856
tt0118421
tt0248654
tt1910272
tt0096697
tt0412142
tt0286486
tt4574334
tt6741278
tt0407362
tt1606375
tt10233448
tt5687612
tt1856010
tt0804503
tt1190634
tt5788792
tt0388629
tt4786824
tt07

### Tv shows we gathered data for

In [5]:
df['ShowName'].unique()

array(['Planet Earth II', 'Planet Earth', 'Breaking Bad',
       'Band of Brothers', 'Chernobyl', 'The Wire', 'Blue Planet II',
       'Cosmos: A Spacetime Odyssey', 'Our Planet',
       'Avatar: The Last Airbender', 'Cosmos', 'The Sopranos',
       'Game of Thrones', 'Rick and Morty', 'The World at War',
       'The Last Dance', 'Life', 'Hagane no renkinjutsushi', 'Sherlock',
       'The Vietnam War', 'Arcane', 'Scam 1992: The Harshad Mehta Story',
       'The Twilight Zone', 'The Beatles: Get Back', 'The Blue Planet',
       'Batman: The Animated Series', 'Human Planet', 'Firefly',
       'Frozen Planet', 'Sahsiyet', "Clarkson's Farm",
       'Death Note: Desu nôto', 'Dekalog', 'True Detective',
       'Attack on Titan', 'The Civil War', 'Kaubôi bibappu',
       'Hunter x Hunter', 'Fargo', 'The Office', 'When They See Us',
       'Africa', 'Apocalypse: La 2ème guerre mondiale',
       'Only Fools and Horses....', 'TVF Pitchers',
       'Last Week Tonight with John Oliver', 'Gravity F

In [12]:
df[df['ShowName'] =='Peaky Blinders']

,Unnamed: 0,Unnamed: 0.1,ID,ShowName,Season,Episode,Name,Date,Rating,Votes
2986,2986,2986.0,tt2442560,Peaky Blinders,1.0,1,Episode #1.1,12 Sep. 2013,8.1,"(9,538)"
2987,2987,2987.0,tt2442560,Peaky Blinders,1.0,2,Episode #1.2,19 Sep. 2013,8.3,"(8,055)"
2988,2988,2988.0,tt2442560,Peaky Blinders,1.0,3,Episode #1.3,26 Sep. 2013,8.3,"(7,523)"
2989,2989,2989.0,tt2442560,Peaky Blinders,1.0,4,Episode #1.4,3 Oct. 2013,8.6,"(7,381)"
2990,2990,2990.0,tt2442560,Peaky Blinders,1.0,5,Episode #1.5,10 Oct. 2013,8.9,"(7,802)"
2991,2991,2991.0,tt2442560,Peaky Blinders,1.0,6,Episode #1.6,17 Oct. 2013,9.2,"(8,599)"
2992,2992,2992.0,tt2442560,Peaky Blinders,2.0,1,Episode #2.1,2 Oct. 2014,8.5,"(6,724)"
2993,2993,2993.0,tt2442560,Peaky Blinders,2.0,2,Episode #2.2,9 Oct. 2014,8.4,"(6,212)"
2994,2994,2994.0,tt2442560,Peaky Blinders,2.0,3,Episode #2.3,16 Oct. 2014,8.7,"(6,080)"
2995,2995,2995.0,tt2442560,Peaky Blinders,2.0,4,Episode #2.4,23 Oct. 2014,8.5,"(5,961)"
